### 정수 인코딩 (Integer Encoding)

In [1]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [21]:
text = "A barber is a person. a barber is good person. \
        a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. \
        Huge secret. His barber kept his word. a barber kept his word. \
        His barber kept his secret. But keeping and keeping such a huge secret \
        to himself was driving the barber crazy. the barber went up a huge mountain."

# sentence tokenization
sentences = sent_tokenize(text)
print("입력 data: ", sentences)

# cleaning ( except stopwords and words length is less than two words)
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
    # word tokenization
    tokenized_sentence = word_tokenize(sentence)
    result = []
    for word in tokenized_sentence:
        word = word.lower() # 소문자화
        if word not in stop_words:
            if len(word) > 2:
                result.append(word)
                if word not in vocab:
                    #빈도수 확인
                    vocab[word] = 0
                vocab[word] += 1
    preprocessed_sentences.append(result)

print("정제된 data: ", preprocessed_sentences)                

입력 data:  ['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret         to himself was driving the barber crazy.', 'the barber went up a huge mountain.']
정제된 data:  [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [22]:
print('빈도수 확인: ', vocab)

빈도수 확인:  {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [23]:
vocab_sorted = sorted(vocab.items(), key = lambda x: x[1], reverse = True) #내림차순
print("높은 순서대로 정렬: ", vocab_sorted)

word_to_idx = {}
i = 0
for (w, freq) in vocab_sorted:
    if freq > 1: #빈도수 작은 단어 제외
        i = i + 1
        word_to_idx[w] = i
print("인덱스 부여: ", word_to_idx)

높은 순서대로 정렬:  [('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]
인덱스 부여:  {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [28]:
# 상위 5개만 사용할 예정
vocab_size = 5
words_frequency = [word for word, idx in word_to_idx.items() if idx >= vocab_size + 1]

#해당 단어에 대한 인덱스 정보를 삭제
for w in words_frequency:
    del word_to_idx[w]
print(word_to_idx)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [29]:
# 단어에 없는 집합에 대한 인덱스 부여
word_to_idx['OOV'] = len(word_to_idx) + 1 

# 정수 인코딩
encoded_sentences = []
for s in preprocessed_sentences:
    encoded_sentence = []
    for w in s:
        try:
            # 단어 집에 있는 단어면 해당 단어의 인덱스(정수) 리턴
            encoded_sentence.append(word_to_idx[w])
        except KeyError:
            # 집합에 없는 단어의 경우 'OOV'의 인덱스 값 리턴
            encoded_sentence.append(word_to_idx['OOV'])
    encoded_sentences.append(encoded_sentence)
print(encoded_sentences)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


In [30]:
from collections import Counter

#하나의 리스트로
all_words_list = sum(preprocessed_sentences, []) 
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [31]:
#counter를 이용하여 빈도수 가져옴
vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [32]:
# 상위 5개만 단어 집합으로 저장
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [34]:
from nltk import FreqDist
import numpy as np

#np.hstack은 문장 구분을 제거해준 것으로 이 전 sum(.,[])과 같은 기능
vocab = FreqDist(np.hstack(preprocessed_sentences))
print(vocab.items())

dict_items([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


### 케라스의 텍스트 전처리

In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [38]:
tokenizer = Tokenizer()

# 빈도수 기준으로 인덱스를 부여한 단어 집합을 생성
tokenizer.fit_on_texts(preprocessed_sentences)
print("인덱스: ", tokenizer.word_index)

# 만약 단어에 대한 빈도수를 확인하고 싶다면
print("빈도수: ", tokenizer.word_counts)

인덱스:  {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
빈도수:  OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [39]:
# 코퍼스에 대해 각 단어를 정수로 변환
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [40]:
# 빈도수가 높은 상위 5개만 사용
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) #1~5번 단어까지 보존할꺼니깐
tokenizer.fit_on_texts(preprocessed_sentences)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [41]:
#'oov' 를 사용하고 싶으면
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

#단, 케라스 토크나이저는 기본적으로 'oov'의 인덱스는 1로 지정
print('index of OOV: {}'.format(tokenizer.word_index['OOV']) )

index of OOV: 1


In [42]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


### 패딩 (Padding)

In [43]:
# Numpy 이용
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

#정수인코딩에서 사용한 예제 그대로 이용
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [44]:
# 문장 중 가장 긴 길이 계산
max_len = max(len(item) for item in encoded)
print("최대 길이 : ", max_len)

최대 길이 :  7


In [46]:
for sentence in encoded:
    while len(sentence) < max_len:
        sentence.append(0)

padded_np = np.array(encoded)
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

In [47]:
#케라스 전처리 도구 이용
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [48]:
padded = pad_sequences(encoded)
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]], dtype=int32)

In [49]:
padded = pad_sequences(encoded, padding='post')
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]], dtype=int32)

In [50]:
padded = pad_sequences(encoded, padding='post', maxlen = 5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]], dtype=int32)

In [52]:
padded = pad_sequences(encoded, padding='post',truncating='post', maxlen = 5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 7,  7,  3,  2, 10],
       [ 1, 12,  3, 13,  0]], dtype=int32)

### 원 핫 인코딩 (One-Hot Encoding

In [56]:
# 한국어 자연어 처리에서의 원핫인코딩
from konlpy.tag import Okt

okt = Okt()
tokens = okt.morphs("안녕 내 취미는 수영하는 거야")
print(tokens)

['안녕', '내', '취미', '는', '수영', '하는', '거야']


In [57]:
word_to_idx = {word : idx for idx, word in enumerate(tokens)}
print("단어 집합: ", word_to_idx)

단어 집합:  {'안녕': 0, '내': 1, '취미': 2, '는': 3, '수영': 4, '하는': 5, '거야': 6}


In [58]:
def one_hot_encoding(word, word_to_idx):
    one_hot_vector = [0]*(len(word_to_idx))
    idx = word_to_idx[word]
    one_hot_vector[idx] = 1
    return one_hot_vector

In [59]:
one_hot_encoding("취미", word_to_idx)

[0, 0, 1, 0, 0, 0, 0]

In [64]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = "오늘 같이 수영하러 갈래? 수영 수업은 아침에 있어 수영 수영 수영" #말 안되는 문장이지만 실습위해 임의로
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('단어 집합 : ', tokenizer.word_index)

단어 집합 :  {'수영': 1, '오늘': 2, '같이': 3, '수영하러': 4, '갈래': 5, '수업은': 6, '아침에': 7, '있어': 8}


In [66]:
sub_text = "아침에 수영 갈래?"
encoded = tokenizer.texts_to_sequences([sub_text])[0] #정수 시퀀스로 변환
print("정수 시퀀스: ", encoded)
one_hot = to_categorical(encoded)
print(one_hot)

정수 시퀀스:  [7, 1, 5]
[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]
